In [ ]:
from DDCASPT2 import DDCASPT2
import pickle, os, shutil
from glob import glob
import numpy as np
from joblib import Parallel, delayed
import pandas as pd
from math import sin, cos, pi
from tqdm.notebook import tqdm
# Geom manipulate
from AaronTools.geometry import Geometry
from tqdm import tqdm
from time import perf_counter
import seaborn as sns

In [ ]:
done = set([i for i in sorted(os.listdir()) if os.path.exists(os.path.join(i,i+".csv"))])

In [ ]:
dirs = []
for i in sorted(os.listdir()):
    try:
        r = float(i)
        dirs.append(i)
    except:
        print("Nope",i)

In [ ]:
radius_range=np.linspace(0.6,3,100)
# radius_range=[0.94]
# chains=np.arange(2,14,2)
chains=np.arange(14,20,2)
print(chains)
# train_ind,test_ind=train_test_split(radius_range, test_size=0.3, random_state=0)
# print(len(train_ind),len(test_ind))
# with open('train_ind.pickle', 'wb') as handle:
#     pickle.dump(train_ind, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('test_ind.pickle', 'wb') as handle:
#     pickle.dump(test_ind, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('test_ind.pickle', 'rb') as handle:
    test_ind = pickle.load(handle)

with open('train_ind.pickle', 'rb') as handle:
    train_ind = pickle.load(handle)
    
print(len(train_ind),len(test_ind))    


In [ ]:
# for i in chains:
topdir = os.getcwd()
basis_set='ANO-RCC-VDZP'


In [ ]:
def gen_run(name,path):
    with open(os.path.join(path,f'run.sh'),'w') as g:
        g.write(f"""#!/bin/bash
#This file is a submission script to request the ISAAC resources from Slurm
#SBATCH --account=ACF-UTK0012            # The project account to be charged
#SBATCH --job-name={name}		       #The name of the job
#SBATCH --nodes=1                     # Number of nodes
#SBATCH --ntasks-per-node=16          # cpus per node
#SBATCH --partition=condo-kvogiatz            # If not specified then default is "campus"
#SBATCH --qos=condo-kvogiatz
#SBATCH --time=1-00:00:00             # Wall time (days-hh:mm:ss)
#SBATCH --error=job.e%J	       # The file where run time errors will be dumped
#SBATCH --output=job.o%J	       # The file where the output of the terminal will be dumped



# PATCH!
# MOVE TO WORKING DIR
echo $SLURM_SUBMIT_DIR
cd $SLURM_SUBMIT_DIR
pwd
# DEFINE WorkDir
mkdir $SLURM_SUBMIT_DIR/$SLURM_JOBID
export WorkDir=$SLURM_SUBMIT_DIR/$SLURM_JOBID
export MOLCAS_WORKDIR="/lustre/isaac/scratch/gjones39"
export MOLCAS="/lustre/isaac/proj/UTK0022/Grier2025/Test/build"
echo $WorkDir



module purge
module load intel-compilers/2021.2.0
module load hdf5/1.10.8-intel
module load cmake/3.23.2-intel

# DEFINE WorkDir
echo "Starting at $(date)"
echo "Running on hosts: $SLURM_NODELIST"
echo "Running on $SLURM_NNODES nodes."
echo "Running $SLURM_NTASKS tasks."
echo "Current working directory is $(pwd)"

# THE COMMAND
pymolcas --new --clean {name}.input -oe {name}.output
# CLEAN-UP AND EXIT
rm -r $SLURM_SUBMIT_DIR/$SLURM_JOBID
echo "Program finished with exit code $? at: $(date)"
""")    

In [ ]:

# topdir = os.getcwd()

topdir = "/lustre/isaac/scratch/gjones39/bigrun"

In [ ]:


def run(i):
    dirname = f'H{i}_chain'
    print(dirname)
    
    if not os.path.exists(dirname):
        os.mkdir(dirname)
        
    first_valid_idx = None  # Track the first valid index
    
    for idxr, r in enumerate(radius_range):
        # Loop radius
        name = f"H{i}_{r:.2f}"
        
        # Create files
        subdirpath = os.path.join(dirname, f'{name}')
        if not os.path.exists(subdirpath):
            os.mkdir(subdirpath)
        
        if not os.path.exists(os.path.join(subdirpath, f'{name}.csv')):
            shutil.rmtree(os.path.join('tmp'), ignore_errors=True)
            
            # Write xyz
            with open(os.path.join(subdirpath, f'{name}.xyz'), 'w') as f:
                f.write(f'{i}\n\n')
                for j in range(i):
                    f.write(f'H {0:>8f} {0:>8f} {j*r:>8f}\n')
            
            print(subdirpath)

            try:
                if first_valid_idx is None:  # First attempt               
                    d = DDCASPT2(subdirpath, basis_set, name, i, i, 0,scf_previous=None,casscf_previous=None,symmetry=1,spin=0,UHF=False,CIROOT=None,frozen=0,pt2maxiter=50,MSroots=1,charge=1,n_jobs=1)(inputwrite=True,run=False,feat=False)
                    first_valid_idx = idxr  # Mark this as the first valid calculation
                    gen_run(name,subdirpath)
                else:
                    previous = os.path.join(
                        topdir,
                        dirname, 
                        f'H{i}_{radius_range[idxr-1]:.2f}', 
                        f"H{i}_{radius_range[idxr-1]:.2f}.RasOrb"
                    )
                    
                    d = DDCASPT2(subdirpath, basis_set, name, i, i, 0,scf_previous=None,casscf_previous=previous,symmetry=1,spin=0,UHF=False,CIROOT=None,frozen=0,pt2maxiter=50,MSroots=1,charge=1,n_jobs=1)(inputwrite=True,run=False,feat=False)
                    gen_run(name,subdirpath)
            except Exception as e:
                print(f"Error at index {idxr} with r={r}: {e}")
                continue  # Skip to the next iteration if it fails
    
    # Cleanup section
    for idxr, r in enumerate(radius_range):
        name = f"H{i}_{r:.2f}" 
        subdirpath = os.path.join(topdir, dirname, f'{name}')
        
        for j in glob(os.path.join(subdirpath, "*GMJ*.csv")) + \
                 glob(os.path.join(subdirpath, "*Orb*")) + \
                 glob(os.path.join(subdirpath, "*h5")) + \
                 glob(os.path.join(subdirpath, "xmldump")):
            os.remove(j)

In [ ]:
for h2n in chains:
    run(h2n)